In [19]:
import matplotlib.pylab as pl
import numpy as np
import scipy as sp
from numpy.linalg import eig
from scipy.io import loadmat
import pdb

In [18]:
pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.9/217.9 kB 188.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 326.0 kB/s eta 0:00:0000:0100:03
Note: you may need to restart the kernel to use updated packages.


In [28]:
def load_data(fname):
    # load the data
    data = loadmat('bcidata.mat')
    # extract images and labels
    X = data['X']
    print(X.shape)
    Y = data['Y']
    print(Y.shape)
    # collapse the time-electrode dimensions
    X = np.reshape(X,(X.shape[0]*X.shape[1],X.shape[2]))
    # transform the labels to (-1,1)
    Y = np.sign((Y[0,:]>0) -.5)
    # pick only first 500 (1000, 3000) datapoints and compare optimal shrinkage
    X = X[:500]
    Y = Y[:500]
    print(X.shape)
    print(Y.shape)
    return X,Y

In [42]:
def crossvalidate_nested(X,Y,f,gammas):
    ''' 
    Optimize shrinkage parameter for generalization performance 
    Input:	X	data (dims-by-samples)
                Y	labels (1-by-samples)
                f	number of cross-validation folds
                gammas	a selection of shrinkage parameters
                trainfunction 	trains linear classifier, returns weight vector and bias term
    '''
    # the next two lines reshape vector of indices in to a matrix:
    # number of rows = # of folds
    # number of columns = # of total data-points / # folds
    N = f*int(np.floor(X.shape[-1]/f))
    idx = np.reshape(np.arange(N),(f,int(np.floor(N/f)))) 
    # pdb.set_trace()
    acc_test = np.zeros((f))
    testgamma = np.zeros((gammas.shape[-1],f))
    print(idx.shape)
    # loop over folds:
    # select one row of 'idx' for testing, all other rows for training
    # call variables (indices) for training and testing 'train' and 'test'
    for ifold in np.arange(f):
        test_set = idx[ifold]
        print(test_set.shape)
        train_set = np.concatenate(idx[0:ifold-1], idx[ifold:])
        print(train_set.shape)
        ...
        ...
        
        # loop over gammas
        for igamma in range(gammas.shape[-1]):
            # each gamma is fed into the inner CV via the function 'crossvalidate_lda'
            # the resulting variable is called 'testgamma'
            testgamma[igamma,ifold] = crossvalidate_lda(...,...,f-1,...)
        # find the the highest accuracy of gammas for a given fold and use it to train an LDA on the training data
        ...
        w,b = train_lda(...,...,...)
        # calculate the accuracy for this LDA classifier on the test data
        ...
        acc_test[ifold] = ...

    # do some plotting
    pl.figure()
    pl.boxplot(testgamma.T)
    pl.xticks(np.arange(gammas.shape[-1])+1,gammas)
    pl.xlabel('$\gamma$')
    pl.ylabel('Accuracy')
    pl.savefig('cv_nested-boxplot.pdf')

    return acc_test,testgamma

<>:46: SyntaxWarning: invalid escape sequence '\g'
<>:46: SyntaxWarning: invalid escape sequence '\g'
/var/folders/c9/f180hbpd3vg432mzv1m0rgs80000gn/T/ipykernel_41018/4199650184.py:46: SyntaxWarning: invalid escape sequence '\g'
  pl.xlabel('$\gamma$')


In [38]:
def crossvalidate_lda(X,Y,f,gamma):
    ''' 
    Test generalization performance of shrinkage lda
    Input:	X	data (dims-by-samples)
                Y	labels (1-by-samples)
                f	number of cross-validation folds
                trainfunction 	trains linear classifier, returns weight vector and bias term
    '''
    N = f*int(np.floor(X.shape[-1]/f))
    idx = np.reshape(np.arange(N),(f,int(np.floor(N/f))))
    acc_test = np.zeros((f))
    
    # loop over folds
    # select one row of idx for testing, all others for training
    # call variables (indices) for training and testing 'train' and 'test'
    for ifold in np.arange(f):
        ...
        ...
        # train LDA classifier with training data and given gamma:
        w,b = train_lda(...,...,...)
        # test classifier on test data:
        ...
        acc_test[ifold] = ...
    return acc_test.mean()

In [39]:
def train_lda(X,Y,gamma):
    '''
    Train a nearest centroid classifier
    '''
    # class means
    mupos = np.mean(X[:,Y>0],axis=1)
    muneg = np.mean(X[:,Y<0],axis=1)

    # inter and intra class covariance matrices
    Sinter = np.outer(mupos-muneg,mupos-muneg)
    #Sinter = np.outer(muneg-mupos,muneg-mupos)
    Sintra = np.cov(X[:,Y>0]) + np.cov(X[:,Y<0])
    # shrink covariance matrix estimate
    Sintra = ...
    # solve eigenproblem
    eigvals, eigvecs = np.linalg.eig(Sinter,Sintra)
    # weight vector
    w = eigvecs[:,eigvals.argmax()]
    # offset
    b = (w.dot(mupos) + w.dot(muneg))/2.
    # return the weight vector
    return w,b

In [44]:
X,Y = load_data('bcidata.mat')
gammas=np.array([0,.005,.05,.5,1])
a,b = crossvalidate_nested(X,Y,10,gammas)
print(a)
print(b)


(5, 62, 5322)
(2, 5322)
(310, 5322)
(500,)
(10, 532)
(532,)


TypeError: only integer scalar arrays can be converted to a scalar index